In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision.models import resnet34

In [2]:
#load the shadow model trained in the other python script
device = "cpu"
resnet_shadow = resnet34(pretrained = False,num_classes = 200).to(device) #resnet_target is the shadow model
resnet_cifar = torch.load("resnet34_shadow_tinyimage_overtrained.pth",map_location=torch.device('cpu'))
resnet_shadow.load_state_dict(resnet_cifar)

/home/ares/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ares/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [3]:
DATA_PATH = 'shadow_tinyimage_resnet.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(
    train_data, batch_size=
     1 , shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1,
                                          shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


KeyboardInterrupt: 

In [ ]:
#generate dataset for attack model
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [ ]:


# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


In [ ]:
dataloader_attack = torch.utils.data.DataLoader(
    
    standardized_data_list, batch_size=32, shuffle=True, num_workers=2) #shuffled training data

In [ ]:
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x



In [ ]:
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Define your training function

def train(model, dataloader, optimizer, criterion, epochs):
    for inputs, labels in dataloader:
        inputs = inputs.float()  # Ensures input tensors are floats
        labels = labels.float().view(-1, 1)  # Ensures labels are floats and reshaped correctly

        optimizer.zero_grad()
        outputs = model(inputs.squeeze(dim = 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

In [ ]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [ ]:
model.train()
train(model,dataloader_attack,optimizer,criterion,20)

Loss: 0.5737922191619873
Loss: 0.557147204875946
Loss: 0.6345616579055786
Loss: 0.51502525806427
Loss: 0.4994433522224426
Loss: 0.5524929165840149
Loss: 0.5721598267555237
Loss: 0.5059529542922974
Loss: 0.49669158458709717
Loss: 0.6722071766853333
Loss: 0.5807992219924927
Loss: 0.5865220427513123
Loss: 0.5485138297080994
Loss: 0.4769091010093689
Loss: 0.544379472732544
Loss: 0.6915156245231628
Loss: 0.6125169992446899
Loss: 0.5605491399765015
Loss: 0.5090764164924622
Loss: 0.6573975086212158
Loss: 0.7256035208702087
Loss: 0.6512302160263062
Loss: 0.6709032654762268
Loss: 0.703004002571106
Loss: 0.6730580925941467
Loss: 0.46448835730552673
Loss: 0.6475969552993774
Loss: 0.4914112389087677
Loss: 0.5193824768066406
Loss: 0.5901832580566406
Loss: 0.5409319400787354
Loss: 0.5819964408874512
Loss: 0.6289772987365723
Loss: 0.5596988797187805
Loss: 0.5031646490097046
Loss: 0.5359064936637878
Loss: 0.5767052173614502
Loss: 0.6527310609817505
Loss: 0.6371476650238037
Loss: 0.5521841049194336
Los

In [ ]:
torch.save(model.state_dict(), 'attack_resnet_tinyimage.pth')

<h1>Evaluation</h1>

In [ ]:
DATA_PATH = "eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=2)
#splitting
for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)


# Define your evaluation function


In [ ]:
dataset = []
resnet_shadow.eval()
with torch.no_grad():
    for images,_, b in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        print(logits)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        

        dataset.append([top_values,b])

tensor([[-10.5248,  -6.0133,  -0.7727,   1.2703,  -1.9050,  -7.3888,  -5.7602,
           0.5817,  -2.0180,  -4.3634,   2.1312,   5.7275, -12.9646,  -8.4753,
          -5.0725,   4.9640,   0.8487,  -6.0237,  -4.4989,  -1.8067,  -1.6913,
          -2.6368,  -0.3992,  -5.8240,  -1.1063,  -8.5472,  -8.2144,  -4.5583,
          -6.1565,  -3.3962,  -9.1987,  -7.8596,  -7.5140,  -5.2720,   0.4879,
           6.8325,  -7.0409,  -3.9132,  -9.3781,  -0.8656,  -5.5044,   0.7781,
          -3.2248,  -6.8028, -13.0102, -12.6748,  -0.4150,  -9.1997,  -1.9617,
          -6.6151,  -0.2209,  -2.3770,  -2.0056,  -5.3372,  -7.7035,  -6.4474,
           5.5262,  -1.2253,  -6.5524,  -8.8351, -15.7582, -12.9315, -16.5205,
         -16.6300,  -8.4703, -17.3953,  -1.7756, -12.7828, -18.9129,  -7.3034,
         -11.1440,  -8.0161, -12.1911,  -9.0672, -10.1508, -11.0548,  -6.5002,
         -13.5913,  -2.7367,  -4.9039,  -1.6464, -17.6493, -11.2556, -11.4849,
         -10.1380,  -9.3809, -17.3015, -14.1617,  -9

In [ ]:
resnet_target_model = resnet34(pretrained = False,num_classes = 200).to(device)
check = torch.load("resnet34_tinyimagenet.pth", map_location=torch.device('cpu'))
resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [ ]:
# Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]

In [ ]:
dataloader_eval = torch.utils.data.DataLoader(
    
    dataset, batch_size=1 , shuffle=False, num_workers=2)

In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = torch.tensor(labels).float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            #predicted = abs(predicted -1)
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Assuming you have a DataLoader called 'test_loader' for evaluation
# Create an instance of your model

# Assuming 'test_loader' is your DataLoader for evaluation
accuracy = evaluate(model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

/tmp/ipykernel_5658/100979714.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).float()


tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tenso

In [ ]:
#1 - 0.26 = 0.74 > 0.68